# Simple Evaluation Function

This notebook can return a static centipawn value for each chess position. It will not look into the future and only consider the current state of the board. 

This method has been developed by Tomasz Michniewski in the Polish chess programming discussion list (progszach).


https://www.chessprogramming.org/Simplified_Evaluation_Function

In [105]:
using Pkg
#Pkg.add("Chess")
using Chess

In [106]:
pawn_square_table = [
    [0,  0,  0,  0,  0,  0,  0,  0],
    [50, 50, 50, 50, 50, 50, 50, 50],
    [10, 10, 20, 30, 30, 20, 10, 10],
    [5,  5, 10, 25, 25, 10,  5,  5],
    [0,  0,  0, 20, 20,  0,  0,  0],
    [5, -5,-10,  0,  0,-10, -5,  5],
    [5, 10, 10,-20,-20, 10, 10,  5],
    [0,  0,  0,  0,  0,  0,  0,  0]
]


knight_square_table = [
    [-50,-40,-30,-30,-30,-30,-40,-50],
    [-40,-20,  0,  0,  0,  0,-20,-40],
    [-30,  0, 10, 15, 15, 10,  0,-30],
    [-30,  5, 15, 20, 20, 15,  5,-30],
    [-30,  0, 15, 20, 20, 15,  0,-30],
    [-30,  5, 10, 15, 15, 10,  5,-30],
    [-40,-20,  0,  5,  5,  0,-20,-40],
    [-50,-40,-30,-30,-30,-30,-40,-50]
]

bishop_square_table = [
    [-20,-10,-10,-10,-10,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5, 10, 10,  5,  0,-10],
    [-10,  5,  5, 10, 10,  5,  5,-10],
    [-10,  0, 10, 10, 10, 10,  0,-10],
    [-10, 10, 10, 10, 10, 10, 10,-10],
    [-10,  5,  0,  0,  0,  0,  5,-10],
    [-20,-10,-10,-10,-10,-10,-10,-20]
]

rook_square_table = [
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 5, 10, 10, 10, 10, 10, 10,  5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [ 0,  0,  0,  5,  5,  0,  0,  0]
]

queen_square_table = [
    [-20,-10,-10, -5, -5,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5,  5,  5,  5,  0,-10],
    [ -5,  0,  5,  5,  5,  5,  0, -5],
    [  0,  0,  5,  5,  5,  5,  0, -5],
    [-10,  5,  5,  5,  5,  5,  0,-10],
    [-10,  0,  5,  0,  0,  0,  0,-10],
    [-20,-10,-10, -5, -5,-10,-10,-20]
]

king_square_middle_game_table = [
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-20,-30,-30,-40,-40,-30,-30,-20],
    [-10,-20,-20,-20,-20,-20,-20,-10],
    [ 20, 20,  0,  0,  0,  0, 20, 20],
    [ 20, 30, 10,  0,  0, 10, 30, 20]
]

king_square_end_game_table = [
    [-50,-40,-30,-20,-20,-30,-40,-50],
    [-30,-20,-10,  0,  0,-10,-20,-30],
    [-30,-10, 20, 30, 30, 20,-10,-30],
    [-30,-10, 30, 40, 40, 30,-10,-30],
    [-30,-10, 30, 40, 40, 30,-10,-30],
    [-30,-10, 20, 30, 30, 20,-10,-30],
    [-30,-30,  0,  0,  0,  0,-30,-30],
    [-50,-30,-30,-30,-30,-30,-30,-50]
]

8-element Vector{Vector{Int64}}:
 [-50, -40, -30, -20, -20, -30, -40, -50]
 [-30, -20, -10, 0, 0, -10, -20, -30]
 [-30, -10, 20, 30, 30, 20, -10, -30]
 [-30, -10, 30, 40, 40, 30, -10, -30]
 [-30, -10, 30, 40, 40, 30, -10, -30]
 [-30, -10, 20, 30, 30, 20, -10, -30]
 [-30, -30, 0, 0, 0, 0, -30, -30]
 [-50, -30, -30, -30, -30, -30, -30, -50]

`get_files()` returns a tuple with all file names as chars.

In [107]:
function get_files()
    return ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h')
end

get_files (generic function with 1 method)

In [108]:
get_files()

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h')

`get_Index_by_File` returns the number according to the file char. 'a' => 1, 'b' => 2, ...

In [109]:
function get_Index_by_File(myfile::Char)
    files=get_files()
    return findfirst(item -> item == myfile, files)
end

get_Index_by_File (generic function with 1 method)

In [110]:
get_Index_by_File('d')

4

More efficient solution for get_Intex_by_file

In [111]:
function get_value(piece::Piece)
    
    if ptype(piece) == PAWN
        return 100
    elseif ptype(piece) == KNIGHT
        return 320
    elseif ptype(piece) == BISHOP
        return 330
    elseif ptype(piece) == ROOK
        return 500
    elseif ptype(piece) == QUEEN
        return 900
    elseif ptype(piece) == KING 
        return 20000
    else
    print("Figur nicht vorhanden")
    return nothing        

    end 
end

get_value (generic function with 1 method)

Additionally we should define where the ending begins. For me it might be either if:

Both sides have no queens or
Every side which has a queen has additionally no other pieces or one minorpiece maximum.

In [112]:
function is_endgame(board)
    # Check if both sides have no queens
    if(isempty(queens(board)))
        return true
    end 
     # Check if each side that has a queen also has no more than one minor piece
    num_white_minor_pieces = squarecount(knights(board, WHITE)) + squarecount(bishops(board, WHITE))
    num_black_minor_pieces = squarecount(knights(board, BLACK)) + squarecount(bishops(board, BLACK))
    if(isempty(rooks(board)) && num_white_minor_pieces <= 1 && num_black_minor_pieces <= 1 )
        return true
    end
    return false
end

is_endgame (generic function with 1 method)

In [113]:
is_endgame(startboard())

false

In [114]:
is_endgame(fromfen("r3k2r/pppppppp/8/8/8/8/PPPPPPPP/R3K2R w KQkq - 0 1"))

true

`get_square_value`...

## Get_Square_Value

The function "get_square_value" is used to evaluate the position of a chess piece on a board. As input parameters it takes a board (board::Board), a chess piece (piece::Piece) and a playing field (square::Square) on which the piece is located. The function calculates a numeric value that describes the strength or weakness of the current position of the piece. 

In [115]:
function get_square_value(board::Board, piece::Piece, square::Square)
    # Bestimmen Sie den Wert des Feldes für diese Schachfigur
    if ptype(piece) == PAWN
        square_table = pawn_square_table
    elseif ptype(piece) == KNIGHT
        square_table = knight_square_table
    elseif ptype(piece) == BISHOP
        square_table = bishop_square_table
    elseif ptype(piece) == ROOK
        square_table = rook_square_table
    elseif ptype(piece) == QUEEN
        square_table = queen_square_table
    elseif ptype(piece) == KING
        if(is_endgame(board))
            square_table = king_square_end_game_table
        else
            square_table = king_square_middle_game_table
        end
    else
        print("Figur nicht vorhanden")
        return nothing
    end
    squareString = tostring(square)
    x = Int(squareString[1])-96
    y = parse(Int64, squareString[2]) 
    if Chess.pcolor(piece) == WHITE
        return square_table[9-y][x]
   elseif Chess.pcolor(piece) == BLACK
        square_table = reverse(square_table)
        return square_table[9-y][x]
    end
end

get_square_value (generic function with 1 method)

In [116]:
function calc_score(board, piece, square)
    return get_value(piece) + get_square_value(board, piece, square)
end 

calc_score (generic function with 3 methods)

In [153]:
function evaluate_position(board:: Board)
    if isterminal(board)
        return terminal_evaluation(board)
    end
    score = 0
    files = get_files()
    for x in 1:8
        for y in 1:8
            square = Square(SquareFile(x),SquareRank(y))
            piece = pieceon(board, square)
            if piece == EMPTY
                continue
            end
            if Chess.pcolor(piece) == WHITE
                score += calc_score(board, piece, square)
            else
                score -= calc_score(board, piece, square)
            end
            println("Piece: " ,piece," Score: ",score, " Square: ", square)
        end
    end 
    return score 
end   

evaluate_position (generic function with 1 method)

In [154]:
b2 = fromfen("rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq - 2 5")
display(b2)
evaluate_position(b2)

Board (rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq -):
 r  n  -  q  k  b  -  r 
 p  p  p  b  -  p  p  p 
 -  -  -  -  p  n  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  P  -  B  -  - 
 -  -  N  -  -  -  -  - 
 P  P  P  Q  P  P  P  P 
 R  -  -  -  K  B  N  R

Piece: PIECE_BR Score: -500 Square: SQ_A8
Piece: PIECE_BP Score: -605 Square: SQ_A7
Piece: PIECE_WP Score: -500 Square: SQ_A2
Piece: PIECE_WR Score: 0 Square: SQ_A1
Piece: PIECE_BN Score: -280 Square: SQ_B8
Piece: PIECE_BP Score: -390 Square: SQ_B7
Piece: PIECE_WP Score: -280 Square: SQ_B2
Piece: PIECE_BP Score: -390 Square: SQ_C7
Piece: PIECE_WN Score: -60 Square: SQ_C3
Piece: PIECE_WP Score: 50 Square: SQ_C2
Piece: PIECE_BQ Score: -845 Square: SQ_D8
Piece: PIECE_BB Score: -1175 Square: SQ_D7
Piece: PIECE_BP Score: -1295 Square: SQ_D5
Piece: PIECE_WP Score: -1175 Square: SQ_D4
Piece: PIECE_WQ Score: -275 Square: SQ_D2
Piece: PIECE_BK Score: -20275 Square: SQ_E8
Piece: PIECE_BP Score: -20375 Square: SQ_E6
Piece: PIECE_WP Score: -20295 Square: SQ_E2
Piece: PIECE_WK Score: -295 Square: SQ_E1
Piece: PIECE_BB Score: -615 Square: SQ_F8
Piece: PIECE_BP Score: -725 Square: SQ_F7
Piece: PIECE_BN Score: -1055 Square: SQ_F6
Piece: PIECE_WB Score: -715 Square: SQ_F4
Piece: PIECE_WP Score: -605 Sq

-5

## Ohne Inkrementelle Evaluation eines Zuges

Sollen wir das Entfernen? 

In [118]:
function evaluate_move(board, move)
    oldScore = evaluate_position(board)
    newBoard = domove(board, move)
    newScore = evaluate_position(board)
    return newScore - oldScore
end

evaluate_move (generic function with 2 methods)

## Inkrementelle Evaluation eines Zuges

### Capurte Piece
The function "subCapturePiece" is used to update the value of a chessboard when a piece is captured by the opponent. As input parameters it takes a chess board (board) and a chess piece that was captured (toMove).

When the function is called, the value of the captured piece is subtracted from the current value of the chess board.

In [141]:
function subCapturePiece(board, square::Square)    
    capturePiece = pieceon(board, square)
    if (capturePiece != EMPTY)
        if Chess.pcolor(capturePiece) == WHITE
            return - calc_score(board, capturePiece, square)
        else
            return calc_score(board, capturePiece, square)
        end
    end
    return 0
end

subCapturePiece (generic function with 2 methods)

In [120]:
function checkEnPassant(board, toMove)
    enpassantSquare = epsquare(board)
    if(enpassantSquare != SQ_NONE && enpassantSquare == toMove)
        lastToMove = to(lastmove(board))
        return subCapturePiece(board, lastToMove)
    end 
    return 0
end 

checkEnPassant (generic function with 1 method)

`calc_castle` checks wether the `move` done was a castleing move. 

If no the score will be returned back.

If yes it will adds the value of the move rook. The moved King will be calculated as the regularmove in the `evaluate_move` function. Casteling Kingside does not effect the value of the rook. Casteling Queenside adds +5 to the value of the position.

Check Castle KingSide is ignored, because the score is not changed. The reason is that the rook on square f1/f8 has the value zero. 

In [122]:
function calc_castle(board, piece, toMove)
    if(ptype(piece) == KING && cancastlequeenside(board, pcolor(piece))  && file(toMove) == FILE_C)
        return  5
    end 
    return 0
end

calc_castle (generic function with 1 method)

In [138]:
function evaluate_move(board:: Board, move:: Move, score)
    if isterminal(domove(board, move))
        return terminal_evaluation(domove(board, move))
    end
    toMove = to(move)
    fromMove = from(move)
    piece = pieceon(board, from(move))
    # Sub Points for old Position from Piece
    score += subCapturePiece(board, from(move))
    score += subCapturePiece(board, toMove)
    if ptype(piece) == PAWN
        score += checkEnPassant(board, toMove)
    end
    if ispromotion(move)
        piece = Piece(sidetomove(board), promotion(move))
    end
    score += calc_castle(board,piece, toMove)
    score += calc_score(board, piece, toMove)
    return score
end

evaluate_move (generic function with 2 methods)

In [139]:
b1 = startboard()
m1 = Move(Square(FILE_E,RANK_2), Square(FILE_E,RANK_4))
evaluate_move(b1, m1, 0) == 40

Capture Piece: PIECE_WPSQ_E2

true

In [ ]:
function terminal_evaluation(board:: Board)
    if ischeckmate(board)
        color = sidetomove(board)
        if color == WHITE
            return -100000
        else
            return 100000
        end
    elseif isstalemate(board)
        return 0
    elseif ismaterialdraw(board)
        return 0
    elseif isrule50draw(board)
        return 0
    end
    return
end

terminal_evaluation (generic function with 1 method)